<a href="https://colab.research.google.com/github/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/visualisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
#
#  Copyright 2024 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck Vidal (UKRI-STFC)

![gVXR](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/Logo-transparent-small.png?raw=1)

# Built-in 3D visualisation

This notebook focuses on the 3D visualisation of the 3D visualisation of the simulation environment. In a nutshell it shares the same simulation code as [first_xray_simulation.ipynb](first_xray_simulation.ipynb).

<div class="alert alert-block alert-warning">
    <b>Note:</b> Make sure the Python packages are already installed. See <a href="../README.md">README.md</a> in the root directory of the repository. If you are running this notebook from Google Colab, please run the cell below to install the package with `!pip install gvxr k3d`
</div>

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install gvxr k3d

# Aims of this session

1. Use K3D to interactively visualise the 3D scene in a Jupyter widget.
2. Control the 3D visualisation features built in gVXR.
3. Take screenshots of the 3D visualisation and save them into PNG files or plot them using Matplotlib.
4. Experiment with the interactive 3D visualisation window.

![](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/visualisation-k3d_screenshot.png?raw=1)

# Main steps

1. Initialise and run the simulation
2. 3D visualisation with K3D in the notebook
3. Static build-in visualisation (in a window if available or using offscreen rendering)
4. Interactive visualisation window if possible

# Cheat Sheet




| Functions | Descriptions |
|-----------|--------------|
| `gvxr.createOpenGLContext` | Create a simulation environment automatically. You must call either `gvxr.createOpenGLContext()` or `gvxr.createNewContext()` before any other functions from gVXR. |
| `gvxr.setSourcePosition` | Set the position of the X-ray source. |
| `gvxr.usePointSource` | Use a point source, i.e. a cone-beam geometry. |
| `gvxr.setMonoChromatic` | Use a monochromatic beam spectrum (i.e. one single energy). |
| `gvxr.setDetectorPosition` | Set the position of the X-ray detector. |
| `gvxr.setDetectorUpVector` | Set the up-vector defining the orientation of the X-ray detector. |
| `gvxr.setDetectorNumberOfPixels` | Set the number of pixels of the X-ray detector. |
| `gvxr.setDetectorPixelSize` | Set the pixel size. Same as the function setDetectorPixelPitch. |
| `gvxr.makeCuboid` | Create a cuboid, centred on (0, 0, 0) and set its label in the scenegraph |
| `gvxr.makeSphere` | Create a sphere and set its label in the scenegraph (i.e. identifier). |
| `gvxr.addPolygonMeshAsOuterSurface` | Add a polygon mesh, given its label, to the X-ray renderer as an outer surface. |
| `gvxr.addPolygonMeshAsInnerSurface` | Add a polygon mesh, given its label, to the X-ray renderer as an inner surface. |
| `gvxr.setElement` | Set the chemical element (or element) corresponding to the material properties. |
| `gvxr.setCompound` | Set the compound corresponding to the material properties of a polygon mesh. |
| `gvxr.setDensity` | Set the density corresponding to the material properties of a polygon mesh. |
| `gvxr.computeXRayImage` | Compute the X-ray projection corresponding to the environment that has previously been set. |
| `gvxr.displayScene` | 3-D visualisation of the 3-D scene (source, detector, and scanned objects). |
| `gvxr.setColour` | Set the colour of a given polygon mesh. |
| `gvxr.takeScreenshot` | Take screenshot. |
| `gvxr.setWindowBackGroundColour` | Set window background colour. |
| `gvxr.setAxisLength` | ***** MISSING ***** |
| `gvxr.getZoom` | ***** MISSING ***** |
| `gvxr.getSceneRotationMatrix` | ***** MISSING ***** |
| `gvxr.setZoom` | ***** MISSING ***** |
| `gvxr.setSceneRotationMatrix` | ***** MISSING ***** |
| `gvxr.displayBeam` | ***** MISSING ***** |
| `gvxr.displayNormalVectors` | ***** MISSING ***** |
| `gvxr.useWireframe` | ***** MISSING ***** |
| `gvxr.renderLoop` | 3-D visualisation of the 3-D scene (source, detector, and scanned objects). |
| `gvxr.terminate` | Close and destroy all the windows and simulation contexts that have been created. No further gVXR's function should be called after `gvxr.terminate`. |

## Import packages

- `os` to create the output directory if needed
- `matplotlib` to show 2D images
- `tifffile` to write TIFF files
- `gvxr` to simulate X-ray images
- `base64` to save a K3D visualisation into a PNG file

In [ ]:
# Import packages
from math import cos, sin, pi
import os # Create the output directory if necessary
import numpy as np # Who does not use Numpy?

import matplotlib # To plot images
import matplotlib.pyplot as plt # Plotting
from matplotlib.colors import LogNorm # Look up table
from matplotlib.colors import PowerNorm # Look up table

font = {'family' : 'serif',
         'size'   : 15
       }
matplotlib.rc('font', **font)

# Uncomment the line below to use LaTeX fonts
# matplotlib.rc('text', usetex=True)

import base64

from gvxrPython3 import gvxr # Simulate X-ray images
from gvxrPython3.utils import visualise # 3D interactive visualisation within the notebook using K3D

Make sure the output directory exists

In [ ]:
output_path = "output_data/visualisation"
if not os.path.exists(output_path):
    os.makedirs(output_path);

# 1. Initialise and run the simulation

All the same as before. Nothing has changed until we look at some visualisation.

In [ ]:
# Create an OpenGL context
print("Create an OpenGL context");
gvxr.createOpenGLContext();

In [ ]:
# Create a source
print("Set up the beam");
gvxr.setSourcePosition(-20.0,  0.0, 0.0, "cm");
gvxr.usePointSource();
#  For a parallel source, use gvxr.useParallelBeam();

In [ ]:
# Set its spectrum, here a monochromatic beam
# 1000 photons of 80 keV (i.e. 0.08 MeV) per ray
gvxr.setMonoChromatic(0.08, "MeV", 1000);
# The following is equivalent: gvxr.setMonoChromatic(80, "keV", 1000);

In [ ]:
# Set up the detector
print("Set up the detector");
gvxr.setDetectorPosition(20.0, 0.0, 0.0, "cm");
gvxr.setDetectorUpVector(0, 0, -1);
gvxr.setDetectorNumberOfPixels(640, 320);
gvxr.setDetectorPixelSize(0.5, 0.5, "mm");

In [ ]:
gvxr.makeCuboid("Cuboid", 5, 4, 3, "cm")
gvxr.makeSphere("Sphere", 15, 15, 1, "cm")

gvxr.addPolygonMeshAsOuterSurface("Cuboid")
gvxr.addPolygonMeshAsInnerSurface("Sphere")

In [ ]:
# Material properties

# Carbon
gvxr.setElement("Sphere", "C");

# Liquid water
gvxr.setCompound("Cuboid", "H2O");
gvxr.setDensity("Cuboid", 1.0, "g/cm3");

In [ ]:
# Compute an X-ray image
# We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.
print("Compute an X-ray image");
x_ray_image = gvxr.computeXRayImage();

# Update the visualisation window
gvxr.displayScene();

# 2. 3D visualisation

It may be useful to visualise the 3D environment to ascertain everything is as expected. 3 different visualisation mode are available.

1. [K3D](https://github.com/K3D-tools/K3D-jupyter),
2. Built-in static visualisation, and
3. Built-in interactive visualisation.

You can control the colour of a given 3D object using
```python
gvxr.setColour(string_ID, red, green, blue, alpha)
```
- `string_ID` is the identifier of the 3D object.
- `red`, `green`, `blue` and `alpha` are values in the range [0, 1].
- `red`, `green` and `blue` control its actual colour.
- `alpha` controls its opacity.

---
## Task:

Change the colour of the cuboid so that it is bright red.

In [ ]:
# %load 'snippets/visualisation-01.py'

## 2.1 [K3D](https://github.com/K3D-tools/K3D-jupyter)

- It is only available in Jupyter notebook. You first need to import the function from the `utils` subpackage.
    ```python
    from gvxrPython3.utils import visualise
    ```
- The function takes 4 optional parameters:
    - `use_log`: Display the X-ray image using a log scale (default: False)
    - `use_negative`: Display the X-ray image in negative (default: False)
    - `sharpen_ksize`: the radius of the Gaussian kernel used in the sharpening filter (default: 1)
    - `sharpen_alpha`: the alpha value used in the sharpening filter (default: 0.0)
- If `sharpen_alpha` is not equal to 0, then the sharpning filter as follows will be used:
    ```python
    # Get the details (the original image minus a blurred version of it).
    # sharpen_ksize controls the amount of blur.
    details = image - gaussian(image, sharpen_ksize)
    
    # Sharpen the image by adding the details back to the original image.
    # sharpen_alpha controls the amount of sharpening.
    sharpened = image + sharpen_alpha * details

    # Preserve the dynamic range
    vmin = np.min(image)
    vmax = np.max(image)
    sharpened[sharpened < vmin] = vmin
    sharpened[sharpened > vmax] = vmax
    ```

---
## Task:

Execute the cell below. In the panel on the right-hand side, go to `Objects->Cuboid` and adjust the opacity to reveal the sphere inside.

In [ ]:
plot = visualise()
if plot:
    plot.display()

### Improving the visualisation of the X-ray image with a log-scale

The X-ray image is not that clear. we can improve it by changing the colour scale and do some image sharpening!

---
### Task:

In the cell below, run the visualisation in negative and using a log-scale. For now, do not use the sharpening image filter. When the visualisation is ready, rotate and zoom in and out to familiarise yourself with the viewer.

In [ ]:
# %load 'snippets/visualisation-02.py'

### Improving the visualisation of the X-ray image with a sharpening filter

The image is much better. Let's see if a bit of image sharpening can further improves the visualisation!

---
### Task:

In the cell below, run the visualisation in negative and using a log-scale and use a filter size of 3, and a apha value of 2 or more. Once it is ready compare the X-ray image with the one above.

In [ ]:
# %load 'snippets/visualisation-03.py'

Some of K3D's code is asynchronous (see: [https://github.com/K3D-tools/K3D-jupyter/blob/main/examples/screenshot_generator.ipynb](https://github.com/K3D-tools/K3D-jupyter/blob/main/examples/screenshot_generator.ipynb). One must make sure the data is ready.

In [ ]:
if plot:
    plot.fetch_screenshot();

<div class="alert alert-block alert-warning">
    <b>Note:</b> Fetching a screenshot is asynchronous. Be patient, wait a few seconds for widgets to synchronise before calling the next cell.
</div>

In [ ]:
if plot:
    data = base64.b64decode(plot.screenshot);
    with open(os.path.join(output_path, "k3d_screenshot.png"), "wb") as fp:
        fp.write(data);
        fp.flush();
        fp.close();

## 2.2 Built-in static visualisation

It is available either
- in a separate window if windowing is enable (e.g. using the `"OPENGL"` backend), or
- in an offscreen framebuffer if windowing is not enable (e.g. using the `"EGL"` backend).

Every time you want to display the visualisation, you may update it with
```python
gvxr.displayScene()
```

To retrieve a screenshot, you must call
```python
screenshot = gvxr.takeScreenshot();
```
It can then be plotted using matplotlib
```python
plt.figure(figsize=(10, 10));
plt.imshow(screenshot);
plt.title("Screenshot of the X-ray simulation environment");
plt.axis('off');
plt.show();
```
or saved into a PNG file
```python
plt.imsave(os.path.join(output_path, "screenshot.png"), np.array(screenshot));
```

In [ ]:
# Update the visualisation
gvxr.displayScene()

# Take a screenshot
screenshot = gvxr.takeScreenshot();

# Display it using Matplotlib
plt.figure(figsize=(10, 10));
plt.imshow(screenshot);
plt.title("Screenshot of the X-ray simulation environment");
plt.axis('off');
plt.show();

# Sve it as a PNG file
plt.imsave(os.path.join(output_path, "screenshot.png"), np.array(screenshot));

### Opacity of a 3D object

When objects made of internal parts are used, it may be useful to reduce the opacity of the outer object to reveal what is inside.

---
### Task:

In the cell below, 

1. using `setColour` again, modify the alpha value of the cuboid colour to reveal the sphere inside (0 for transparent, 1 for opaque),
2. update the visualisation,
3. take a screenshot, and
4. display it using Matplotlib.

<div class="alert alert-block alert-warning">
    <b>Note:</b> As the beam is shown in pink with transparency and the cuboid is red, a low value of alpha is expected, e.g. below 0.3 in this case
</div>

In [ ]:
# %load 'snippets/visualisation-04.py'

### Window background colour

When I want to include a screenshot in an article, I often change the window background colour to white. It can be done with
```python
gvxr.setWindowBackGroundColour(red, green, blue)
```
`red`, `green`, `blue` and `alpha` are values in the range [0, 1].

---
### Task:

In the cell below, 

1. modify the window background colour to any colour of your choice,
2. update the visualisation,
3. take a screenshot, and
4. display it using Matplotlib.

In [ ]:
# %load 'snippets/visualisation-05.py'

### Display the origin of the workd

By default the origin of the workd is shown 3 axes (X in red, Y in green and Z in blue). If they are not visible, we can increase their length using `gvxr.setAxisLength`.

In [ ]:
# Retrieve the bounding box of the cuboid
x_min, y_min, z_min, x_max, y_max, z_max = gvxr.getNodeAndChildrenBoundingBox("Cuboid", "cm")
x_range = x_max - x_min
y_range = y_max - y_min
z_range = z_max - z_min

# Increase the size of the axes
gvxr.setAxisLength(1.5 * max(x_range, y_range, z_range), "cm")

# Update the visualisation
gvxr.displayScene()

# Take a screenshot
screenshot = gvxr.takeScreenshot();

# Display it using Matplotlib
plt.figure(figsize=(10, 10));
plt.imshow(screenshot);
plt.title("Screenshot of the X-ray simulation environment");
plt.axis('off');
plt.show();

### Zooming in/out and rotating the 3D view

Two main sates control the 3D view. A zooming variable and a 4x4 transformation matrix. The zoom is set automatically when the visualisation is first updated. The transformation matrix is a identy matrix. You may retrieve their current state with:

```python
gvxr.getZoom()
gvxr.getSceneRotationMatrix()
```

---
### Task:

In the cell below, print

1. the value of the zoom parameter, and
2. transformation matrix.

In [ ]:
# %load 'snippets/visualisation-06.py'

You may modify their values using:

```python
gvxr.setZoom(value)
gvxr.setSceneRotationMatrix([
    a, b, c, c,
    e, f, g, h, 
    i, j, k, l,
    m, n, o, p]);
```

---
### Task:

In the cell below, 

1. modify the zoom value until the cuboid occupies most of the visualisation (use a smaller value than the one printed in the last cell you executed),
2. turn off the visualisation of the beam using `gvxr.displayBeam(False)`
3. update the visualisation,
4. take a screenshot, and
5. display it using Matplotlib.

In [ ]:
# %load 'snippets/visualisation-07.py'

---
### Task:

In the cell below, 

1. turn the 3D view around its Y-axis by 30 degrees. The matrix is
   $\left[\begin{array}{cccc}
   \cos(\theta) & 0 & \sin(\theta) & 0	\\
   0 & 1 & 0 & 0	\\
   -\sin(\theta) & 0 & \cos(\theta) & 0	\\
   0 & 0 & 0 & 1
   \end{array}\right]$
3. update the visualisation,
4. take a screenshot, and
5. display it using Matplotlib.

In [ ]:
# %load 'snippets/visualisation-08.py'

### Other functions controling the visualisation

- Display/Hide the detector in the 3D visualisation: `gvxr.displayDetector(True/False)`
- Display/Hide the normal vectors of the sample in the 3D visualisation: `gvxr.displayNormalVectors(True/False)`
- Show the 3D objects in wireframe mode: `gvxr.useWireframe(True/False)`
- Enable/Disable lighting: `gvxr.useLighing(True/False)`
- Show the X-ray image as a negative or positive image: `gvxr.useNegative(True/False)`

---
### Task:

In the cell below, 

1. show the normal vectors of the sample in the 3D visualisation
2. display the 3D objects in wireframe mode
3. update the visualisation,
4. take a screenshot, and
5. display it using Matplotlib.


In [ ]:
# %load 'snippets/visualisation-09.py'

## 2.3 Built-in interactive visualisation:

- It is only available if windowing is enable (e.g. using the `"OPENGL"` backend)
- The user can rotate the 3D scene and zoom-in and -out in the visualisation window.
- Keys are:
    - Q/Escape: to quit the event loop (does not close the window)
    - B: display/hide the X-ray beam
    - W: display the polygon meshes in solid or wireframe
    - N: display the X-ray image in negative or positive
    - H: display/hide the X-ray detector
- Mouse interactions:
    - Zoom in/out: mouse wheel
    - Rotation: Right mouse button down + move cursor
- To activate the interactive visualisation mode call:
  ```python
  gvxr.renderLoop();
  ```
  Note that the call stops the execution of other functions. You must 1st exit the interactive visualisation mode before calling further Python functions.
- To exit the interactive visualisation mode press `<Q>` or `<ESC>`, or close the window.

---
### Task:

In the cell below, 

1. execute `gvxr.renderLoop()`
2. turn on the beam visualisation by pressing `<B>`
3. turn off the wireframe mode by pressing `<W>`
4. rotate and zoom in so that you get a nice view of the simulation
5. exit the interactive visualisation mode press `<Q>` or `<ESC>`, or close the window
6. print
    1. the new value of the zoom parameter, and
    2. transformation matrix (it must be much more complex now).

In [ ]:
gvxr.renderLoop()
print("Zoom:", gvxr.getZoom())
print("Transformation matrix:", gvxr.getSceneRotationMatrix())

# Cleaning up

Once we have finished, it is good practice to clean up the OpenGL contexts and windows with the following command. Note that due to the object-oriented programming nature of the core API of gVXR, this step is automatic anyway.

In [ ]:
# Works on Windows (gvxr >= 2.0.9)
# Will work on Linux (gvxr >= 2.0.10)
if os.name == 'nt':
    gvxr.destroy(); 
# Does not work on Windows.
# Deprecated, for backward compatibility on Linux.
else:
    gvxr.terminate(); 